In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import datetime
import librosa
import IPython.display as ipd
import math
import librosa.display
import sklearn
import seaborn as sns
import scipy.signal as sig
import warnings
import scipy.io.wavfile as siow
import matplotlib.colors as color
#suppress depricate warnings
warnings.filterwarnings("ignore")
#load annotations file
X = pd.read_csv(r'..\annotations\training.txt', sep="\t", header=0)


In [ ]:
X['High Freq (Hz)'].describe()

In [ ]:
#hear the wav files at the training data time stamps
def hear(file_path):
    ref_table=X[X['filename']==file_path]
    num_rows= np.shape(X[X['filename']==file_path])[0]
    t_start= ref_table[ ref_table.columns[2]][0:num_rows+1]
    t_end= ref_table[ ref_table.columns[2]][0:num_rows+1]+ ref_table[ ref_table.columns[4]][0:num_rows+1]
    for i,j in zip(t_start,t_end):
        ipd.display(ipd.Audio(samples[math.floor(i)*sampling_rate:math.ceil(j)*sampling_rate],rate=sampling_rate))

In [ ]:
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)


In [ ]:
#Scipy plot for single spectrogram and Periodogram Side for direct comparison
def spec_plot2_one(file_path,num):
    [samp, data] = siow.read(file_path, mmap=False)
    num_rows= np.shape(X[X['filename']==file_path])[0]
    ref_table=X[X['filename']==file_path]
    temp=math.ceil(np.sqrt(num_rows))
  
    t_start=np.array(ref_table[ref_table.columns[2]])[num]
    t_end=np.array(ref_table[ref_table.columns[2]])[num]+np.array(ref_table[ref_table.columns[4]])[num]
    
  
    s=data[math.floor(t_start)*samp:math.ceil(t_end)*samp]
        #filter data to spectral bands
    [b,a] = sig.butter(4,[10/samp, 100/samp],'bandpass','ba') 
    data1 = sig.filtfilt(b, a, s)
        #normalize
    data_norm=normalize(data1)
    
    
    fig, axs = plt.subplots(nrows=1, ncols=2,figsize=(20, 5), facecolor='w', edgecolor='k')
    #plot spectrogram on lower axis
        
    [f, t, Sxx] = sig.spectrogram(data_norm, samp,'hann',nperseg=samp,noverlap=samp*0.9)
    cmap = plt.get_cmap('magma')
    norm = color.Normalize(vmin=np.amin(Sxx), vmax=np.amax(Sxx))
    
    axs[0].pcolormesh(t, f, Sxx, cmap=cmap, norm=norm)  
    axs[0].set_ylim(10,70)
    
  
    axs[1].plot(f,Sxx)
    axs[1].set_xlim(0,60)
    plt.show()
    return f,t,Sxx

In [ ]:
#Scipy plot for all spectrograms and periodograms for a given wav 
def spec_plot2_all(file_path):
    a=X[X['filename']=='nn06f_20180806_000000.wav']['Selection']
    b=a-np.min(a)
    np.vectorize(spec_plot2_one)('nn06f_20180806_000000.wav',b)

In [ ]:
spec_plot2_all('nn06f_20180806_000000.wav')

In [ ]:
#Plots the spectrogram and power between any random two time points in the wav file
#useful to see what the spectrogram looks like when an elephant
#signal is not present (not the labeled cases)

def spec_any_plot_one(file_path,t_start,t_end):
    [samp, data] = siow.read(file_path, mmap=False)
  
    fig, axs = plt.subplots(nrows=1, ncols=2,figsize=(20, 5), facecolor='w', edgecolor='k')
    
    s=data[math.floor(t_start)*samp:math.ceil(t_end)*samp]
        #filter data to spectral bands
    [b,a] = sig.butter(4,[10/samp, 100/samp],'bandpass','ba') 
    data1 = sig.filtfilt(b, a, s)
        #normalize
    data_norm=normalize(data1)
    
    #plot spectrogram on lower axis
        
    [f, t, Sxx] = sig.spectrogram(data_norm, samp,'hann',nperseg=samp,noverlap=samp*0.9)
    cmap = plt.get_cmap('magma')
    norm = color.Normalize(vmin=np.amin(Sxx), vmax=np.amax(Sxx))
   
    axs[0].pcolormesh(t, f, Sxx, cmap=cmap, norm=norm)  
    axs[0].set_ylim(10,70)
    
    axs[1].plot(f,Sxx)
    axs[1].set_xlim(0,60)

    plt.show()
    
    return f,t,Sxx

#Divides signal into plots of time length "0.1*(t_end-t_start)"
#and then plots thespectrogram and power between the two time points 
#in the wav file useful to see what the spectrogram looks like 
#when an elephant signal is not present. More granular than spec_any_plot_one
def spec_any_plot_all(file_path,t_start,t_end):

    ts=np.arange(t_start, t_end, 0.1*(t_end-t_start))
    te=np.arange(t_start+10, t_end+10, 0.1*(t_end-t_start))
    d = {'a': ts, 'b': te}
    df = pd.DataFrame(data=d)
    np.vectorize(spec_any_plot_one)(file_path,df['a'],df['b'])

In [ ]:
#Example
spec_any_plot_all('nn06f_20180806_000000.wav',40000,40100)